In [1]:
# | default_exp _testing.in_memory_broker

In [2]:
# | export
import uuid
from collections import namedtuple
from dataclasses import dataclass
from contextlib import contextmanager
import inspect
import asyncio
import copy

from typing import *
import fastkafka._application.app
import fastkafka._components.aiokafka_consumer_loop
import fastkafka._components.aiokafka_producer_manager
from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from aiokafka.structs import ConsumerRecord, TopicPartition, RecordMetadata

import fastkafka._application.app
from fastkafka._components.meta import copy_func, patch, delegates, classcontextmanager
from fastkafka._components.logger import get_logger

In [3]:
from contextlib import asynccontextmanager
import unittest

import pytest

from fastkafka.testing import ApacheKafkaBroker

In [4]:
# | export

logger = get_logger(__name__)

# Local Kafka broker
> In-memory mockup of Kafka broker protocol

In [5]:
# | export


def create_consumer_record(topic: str, msg: bytes) -> ConsumerRecord: # type: ignore
    record = ConsumerRecord(
        topic=topic,
        partition=0,
        offset=0,
        timestamp=0,
        timestamp_type=0,
        key=None,
        value=msg,
        checksum=0,
        serialized_key_size=0,
        serialized_value_size=0,
        headers=[],
    )
    return record

In [6]:
record = create_consumer_record("my_topic", b"my_msg")
record.partition = 1
assert record.partition == 1

In [7]:
# | export


@dataclass
class ConsumerMetadata:
    topic: str
    offset: int

In [8]:
consumer_meta = ConsumerMetadata("my_topic", 0)
assert consumer_meta.topic == "my_topic"
assert consumer_meta.offset == 0

In [9]:
# | export


# InMemoryBroker
@classcontextmanager()
class InMemoryBroker:
    def __init__(self, topics: Set[str]):
        self.data: Dict[str, List[ConsumerRecord]] = {topic: list() for topic in topics}  # type: ignore
        self.consumers_metadata: Dict[uuid.UUID, List[ConsumerMetadata]] = {}
        self.is_started: bool = False

    def connect(self) -> uuid.UUID:
        return uuid.uuid4()

    def subscribe(self, actor_id: uuid.UUID, *, auto_offest_reset: str, topic: str) -> None:
        consumer_metadata = self.consumers_metadata.get(actor_id, list())
        # todo: what if whatever?
        consumer_metadata.append(
            ConsumerMetadata(
                topic, len(self.data[topic]) if auto_offest_reset == "latest" else 0
            )
        )
        self.consumers_metadata[actor_id] = consumer_metadata

    def unsubscribe(self, actor_id: uuid.UUID) -> None:
        try:
            del self.consumers_metadata[actor_id]
        except KeyError:
            logger.warning(f"No subscription with {actor_id=} found!")

    def produce(  # type: ignore
        self, *, topic: str, msg: bytes, key: Optional[bytes] = None
    ) -> RecordMetadata:
        if topic in self.data:
            record = create_consumer_record(topic, msg)
            self.data[topic].append(record)
        else:
            # todo: log only once
            logger.warning(
                f"Topic {topic} is not available during auto-create initialization"
            )
        return RecordMetadata(
            topic=topic,
            partition=0,
            topic_partition=TopicPartition(topic=topic, partition=0),
            offset=0,
            timestamp=1680602752070,
            timestamp_type=0,
            log_start_offset=0,
        )

    def consume(  # type: ignore
        self, actor_id: uuid.UUID
    ) -> Dict[TopicPartition, List[ConsumerRecord]]:
        msgs: Dict[TopicPartition, List[ConsumerRecord]] = {}  # type: ignore

        consumer_metadata = self.consumers_metadata[actor_id]

        for metadata in consumer_metadata:
            try:
                msgs[TopicPartition(metadata.topic, 0)] = self.data[metadata.topic][
                    metadata.offset :
                ]
                metadata.offset = len(self.data[metadata.topic])
            except KeyError:
                raise RuntimeError(
                    f"{metadata.topic=} not found, did you pass it to InMemoryBroker on init to be created?"
                )
        return msgs

    @contextmanager
    def lifecycle(self) -> Iterator["InMemoryBroker"]:
        raise NotImplementedError()

    async def _start(self) -> str:
        logger.info("InMemoryBroker._start() called")
        self.__enter__() # type: ignore
        return "localbroker:0"

    async def _stop(self) -> None:
        logger.info("InMemoryBroker._stop() called")
        self.__exit__(None, None, None) # type: ignore

In [10]:
in_memory_broker = InMemoryBroker(["my_topic"])

actor_id = in_memory_broker.connect()

with pytest.raises(KeyError) as e:
    in_memory_broker.consume(actor_id)
    
print(e)

in_memory_broker.subscribe(actor_id=actor_id, auto_offest_reset="earliest", topic="my_topic")

msg = in_memory_broker.consume(actor_id)
assert msg == {TopicPartition(topic='my_topic', partition=0): []}, msg

# with raise()
in_memory_broker.produce(topic="not_my_topic", msg=b"not my message")

<ExceptionInfo KeyError(UUID('b47b4815-c963-4bd9-97a4-c69c7780c140')) tblen=2>
[WARNING] __main__: Topic not_my_topic is not available during auto-create initialization


RecordMetadata(topic='not_my_topic', partition=0, topic_partition=TopicPartition(topic='not_my_topic', partition=0), offset=0, timestamp=1680602752070, timestamp_type=0, log_start_offset=0)

In [11]:
#| notest

with ApacheKafkaBroker(["my_topic"], apply_nest_asyncio=True) as bootstrap_servers:
    producer = AIOKafkaProducer(bootstrap_servers=bootstrap_servers)
    await producer.start()
    for _ in range(1000):
        record = await producer.send(topic="not_my_topic", value=b"not my message")
    await producer.stop()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.sta

In [12]:
#| notest

with ApacheKafkaBroker(["my_topic"], apply_nest_asyncio=True) as bootstrap_servers:
    consumer = AIOKafkaConsumer("my_topic", bootstrap_servers=bootstrap_servers)
    await consumer.start()
    print("getmany()...")
    msg = await consumer.getmany(timeout_ms=0)
    print("exiting...")
    await consumer.stop()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] aiokafka.consumer.subscription_state: Updating subscri

## Consumer patching

We need to patch AIOKafkaConsumer methods so that we can redirect the consumer to our local kafka broker.

Patched methods:

- [x] \_\_init\_\_
- [x] start
- [x] subscribe
- [x] stop
- [x] getmany

In [23]:
# | export


# InMemoryConsumer
class InMemoryConsumer:
    def __init__(
        self,
        broker: InMemoryBroker,
    ) -> None:
        logger.info("AIOKafkaConsumer patched __init__() called()")
        self.broker = broker
        self._id: Optional[uuid.UUID] = None
        self._auto_offset_reset: str = "latest"

    @delegates(AIOKafkaConsumer)
    def __call__(
        self, 
        **kwargs: Any
    ) -> "InMemoryConsumer":        
        # todo: function
        defaults = {k: v.default for k, v in inspect.signature(InMemoryConsumer.__call__).parameters.items()}
        defaults.update(kwargs)
        kwargs_with_defaults = defaults
        
        consume_copy = InMemoryConsumer(self.broker)
        consume_copy._auto_offset_reset = kwargs_with_defaults["auto_offset_reset"]
        return consume_copy

    @delegates(AIOKafkaConsumer.start)
    async def start(self, **kwargs: Any) -> None:
        pass

    @delegates(AIOKafkaConsumer.stop)
    async def stop(self, **kwargs: Any) -> None:
        pass

    @delegates(AIOKafkaConsumer.subscribe)
    def subscribe(self, topics: List[str], **kwargs: Any) -> None:
        raise NotImplementedError()
        
    @delegates(AIOKafkaConsumer.getmany)
    async def getmany( # type: ignore
        self, **kwargs: Any
    ) -> Dict[TopicPartition, List[ConsumerRecord]]:
        raise NotImplementedError()

In [24]:
broker = InMemoryBroker(["topic"])

ConsumerClass = InMemoryConsumer(broker)

for cls in [ConsumerClass, AIOKafkaConsumer]:

    consumer = cls()
    assert consumer._auto_offset_reset == "latest"

    consumer = cls(auto_offset_reset="earliest")
    assert consumer._auto_offset_reset == "earliest"

    consumer = cls(auto_offset_reset="whatever")
    assert consumer._auto_offset_reset == "whatever"
    
    await consumer.stop()
    

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
{}
{'self': <class 'inspect._empty'>, 'loop': None, 'bootstrap_servers': 'localhost', 'client_id': 'aiokafka-0.8.0', 'group_id': None, 'key_deserializer': None, 'value_deserializer': None, 'fetch_max_wait_ms': 500, 'fetch_max_bytes': 52428800, 'fetch_min_bytes': 1, 'max_partition_fetch_bytes': 1048576, 'request_timeout_ms': 40000, 'retry_backoff_ms': 100, 'auto_offset_reset': 'latest', 'enable_auto_commit': True, 'auto_commit_interval_ms': 5000, 'check_crcs': True, 'metadata_max_age_ms': 300000, 'partition_assignment_strategy': (<class 'kafka.coordinator.assignors.roundrobin.RoundRobinPartitionAssignor'>,), 'max_poll_interval_ms': 300000, 'rebalance_timeout_ms': None, 'session_timeout_ms': 10000, 'heartbeat_interval_ms': 3000, 'consumer_timeout_ms': 200, 'max_poll_records': None, 'ssl_context': None, 'security_protocol': 'PLAINTEXT', 'api_version': 'auto', 'exclude_internal_topics': True, 'connections_max_idle_ms': 540000, '

Patching start so that we don't try to start the real AIOKafkaConsumer instance

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.start)
async def start(self: InMemoryConsumer, **kwargs: Any) -> None:
    logger.info("AIOKafkaConsumer patched start() called()")
    if self._id is not None:
        raise RuntimeError(
            "Consumer start() already called! Run consumer stop() before running start() again"
        )
    self._id = self.broker.connect()

In [ ]:
broker = InMemoryBroker(["my_topic"])

ConsumerClass = InMemoryConsumer(broker)

for cls in [ConsumerClass]:

    consumer = cls()
    await consumer.start()
    await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()


Patching subscribe so that we can connect to our Local, in-memory, Kafka broker

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.subscribe)
def subscribe(self: InMemoryConsumer, topics: List[str], **kwargs: Any) -> None:
    logger.info("AIOKafkaConsumer patched subscribe() called")
    if self._id is None:
        raise RuntimeError("Consumer start() not called! Run consumer start() first")
    logger.info(f"AIOKafkaConsumer.subscribe(), subscribing to: {topics}")
    for topic in topics:
        self.broker.subscribe(
            self._id, topic=topic, auto_offest_reset=self._auto_offset_reset
        )

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass()

await consumer.start()
consumer.subscribe(["my_topic"])

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']


Patching stop so that be dont break anything by calling the real AIOKafkaConsumer stop()

In [ ]:
# | export

@patch
@delegates(AIOKafkaConsumer.stop)
async def stop(self: InMemoryConsumer, **kwargs: Any) -> None:
    logger.info("AIOKafkaConsumer patched stop() called")
    if self._id is None:
        raise RuntimeError(
            "Consumer start() not called! Run consumer start() first"
        )
    self.broker.unsubscribe(self._id)

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])
    
ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass()

await consumer.start()
consumer.subscribe(["my_topic"])
await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called


Patching getmany so that the messages are pulled from our Local, in-memory, Kafka broker

In [ ]:
# | export

@patch
@delegates(AIOKafkaConsumer.getmany)
async def getmany( # type: ignore
    self: InMemoryConsumer, **kwargs: Any
) -> Dict[TopicPartition, List[ConsumerRecord]]:
    return self.broker.consume(self._id) # type: ignore

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass(auto_offset_reset="latest")

await consumer.start()

consumer.subscribe(["my_topic"])
await consumer.getmany()

await consumer.stop()


[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called


## Producer patching

We need to patch AIOKafkaProducer methods so that we can redirect the producer to our local kafka broker

- [x] \_\_init\_\_
- [x] start
- [x] stop
- [x] send

In [ ]:
# | export


class InMemoryProducer:
    def __init__(self, broker: InMemoryBroker, **kwargs: Any) -> None:
        logger.info("AIOKafkaProducer patched __init__() called()")
        self.broker = broker
        self.id: Optional[uuid.UUID] = None

    @delegates(AIOKafkaProducer)
    def __call__(self, **kwargs: Any) -> "InMemoryProducer":
        return InMemoryProducer(self.broker)

    @delegates(AIOKafkaProducer.start)
    async def start(self, **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.stop)
    async def stop(self, **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.send)
    async def send(  # type: ignore
        self: AIOKafkaProducer,
        topic: str,
        msg: bytes,
        key: Optional[bytes] = None,
        **kwargs: Any,
    ):
        raise NotImplementedError()

Patching AIOKafkaProducer start so that we mock the startup procedure of AIOKafkaProducer

In [ ]:
# | export


@patch # type: ignore
@delegates(AIOKafkaProducer.start)
async def start(self: InMemoryProducer, **kwargs: Any) -> None:
    logger.info("AIOKafkaProducer patched start() called()")
    if self.id is not None:
        raise RuntimeError(
            "Producer start() already called! Run producer stop() before running start() again"
        )
    self.id = self.broker.connect()

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()

[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched start() called()


Patching AIOKafkaProducerStop so that we don't uniintentionally try to stop a real instance of AIOKafkaProducer

In [ ]:
# | export


@patch # type: ignore
@delegates(AIOKafkaProducer.stop)
async def stop(self: InMemoryProducer, **kwargs: Any) -> None:
    logger.info("AIOKafkaProducer patched stop() called")
    if self.id is None:
        raise RuntimeError(
            "Producer start() not called! Run producer start() first"
        )

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()
await producer.stop()

[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaProducer patched stop() called


Patching AIOKafkaProducer send so that we redirect sent messages to Local, in-memory, Kafka broker

In [ ]:
# | export


@patch
@delegates(AIOKafkaProducer.send)
async def send( # type: ignore
    self: InMemoryProducer,
    topic: str,
    msg: bytes,
    key: Optional[bytes] = None,
    **kwargs: Any,
): #asyncio.Task[ConsumerRecord]
    if self.id is None:
        raise RuntimeError(
            "Producer start() not called! Run producer start() first"
        )
    record = self.broker.produce(topic=topic, msg=msg, key=key)

    async def _f(record: ConsumerRecord = record) -> ConsumerRecord: # type: ignore
        return record

    return asyncio.create_task(_f())

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()
msg_fut = await producer.send("my_topic", b"some_msg")
await msg_fut

[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched start() called()


RecordMetadata(topic='my_topic', partition=0, topic_partition=TopicPartition(topic='my_topic', partition=0), offset=0, timestamp=1680602752070, timestamp_type=0, log_start_offset=0)

## Add patching to InMemoryBroker

In [ ]:
# | export


@patch
@contextmanager
def lifecycle(self: InMemoryBroker) -> Iterator[InMemoryBroker]:
    logger.info(
        "InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!"
    )
    try:
        logger.info("InMemoryBroker starting")
        
        old_consumer_app = fastkafka._application.app.AIOKafkaConsumer
        old_producer_app = fastkafka._application.app.AIOKafkaProducer
        old_consumer_loop = fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer
        old_producer_manager = fastkafka._components.aiokafka_producer_manager.AIOKafkaProducer
        
        fastkafka._application.app.AIOKafkaConsumer = InMemoryConsumer(self)
        fastkafka._application.app.AIOKafkaProducer = InMemoryProducer(self)
        fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer = InMemoryConsumer(self)
        fastkafka._components.aiokafka_producer_manager.AIOKafkaProducer = InMemoryProducer(self)
        
        self.is_started = True
        yield self
    finally:
        logger.info("InMemoryBroker stopping")
        
        fastkafka._application.app.AIOKafkaConsumer = old_consumer_app
        fastkafka._application.app.AIOKafkaProducer = old_producer_app
        fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer = old_consumer_loop
        fastkafka._components.aiokafka_producer_manager.AIOKafkaProducer = old_producer_manager
        
        self.is_started = False

In [ ]:
assert fastkafka._application.app.AIOKafkaConsumer == AIOKafkaConsumer
assert fastkafka._application.app.AIOKafkaProducer == AIOKafkaProducer

with InMemoryBroker(["topic"]) as broker:
    assert isinstance(fastkafka._application.app.AIOKafkaConsumer, InMemoryConsumer)
    assert isinstance(fastkafka._application.app.AIOKafkaProducer, InMemoryProducer)
    assert fastkafka._application.app.AIOKafkaConsumer().broker == broker
    assert fastkafka._application.app.AIOKafkaProducer().broker == broker
    
assert fastkafka._application.app.AIOKafkaConsumer == AIOKafkaConsumer
assert fastkafka._application.app.AIOKafkaProducer == AIOKafkaProducer

<class '__main__.InMemoryBroker'>.__enter__
[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
<class '__main__.InMemoryBroker'>.__exit__
[INFO] __main__: InMemoryBroker stopping


## Broker, consumer and producer integration tests

In [ ]:
@asynccontextmanager
async def create_consumer_and_producer(
    auto_offset_reset: str = "latest",
) -> AsyncIterator[Tuple[AIOKafkaConsumer, AIOKafkaProducer]]:
    consumer = fastkafka._application.app.AIOKafkaConsumer(auto_offset_reset=auto_offset_reset)
    producer = fastkafka._application.app.AIOKafkaProducer()

    await consumer.start()
    await producer.start()

    yield (consumer, producer)

    await consumer.stop()
    await producer.stop()

In [ ]:
def checkEqual(L1, L2):
    return len(L1) == len(L2) and sorted(L1) == sorted(L2)

In [ ]:
assert checkEqual([1, 2], [3]) == False
assert checkEqual([1, 2, 3], [3, 2, 1]) == True

Sanity check, let's see if the messages are sent to broker and received by the consumer

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(320)]

with InMemoryBroker([topic]) as broker:
    async with create_consumer_and_producer(auto_offset_reset="earliest") as (
        consumer,
        producer,
    ):
        [await producer.send(topic, msg) for msg in sent_msgs]
        consumer.subscribe([topic])
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]
        data = [msg.value for msg in broker.data[topic]]
    assert checkEqual(
        received_msgs, sent_msgs
    ), f"{sent_msgs=}\n{received_msgs=}\n{data=}"

<class '__main__.InMemoryBroker'>.__enter__
[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
<class '__main__.InMemoryBroker'>.__exit__
[INFO] __main__: InMemoryBroker

Check if only subscribed topic messages are received by the consumer

In [ ]:
topic1 = "test_topic1"
topic2 = "test_topic2"
sent_msgs_1 = [(f"msg{i}" + topic1).encode("UTF-8") for i in range(32)]
sent_msgs_2 = [(f"msg{i}" + topic2).encode("UTF-8") for i in range(32)]

with InMemoryBroker([topic1, topic2]) as broker:
    async with create_consumer_and_producer(auto_offset_reset="earliest") as (
        consumer,
        producer,
    ):
        [await producer.send(topic1, msg) for msg in sent_msgs_1]
        [await producer.send(topic2, msg) for msg in sent_msgs_2]

        consumer.subscribe([topic1])
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]

    assert checkEqual(sent_msgs_1, received_msgs)

<class '__main__.InMemoryBroker'>.__enter__
[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic1']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
<class '__main__.InMemoryBroker'>.__exit__
[INFO] __main__: InMemoryBroke

Check if msgs are received only after subscribing when auto_offset_reset is set to "latest"

In [ ]:
topic = "test_topic"
sent_msgs_before = [f"msg{i}".encode("UTF-8") for i in range(32)]
sent_msgs_after = [f"msg{i}".encode("UTF-8") for i in range(32, 64)]

with InMemoryBroker([topic]) as broker:
    async with create_consumer_and_producer() as (consumer, producer):
        [await producer.send(topic, msg) for msg in sent_msgs_before]

        consumer.subscribe([topic])
        [await producer.send(topic, msg) for msg in sent_msgs_after]
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]

    assert checkEqual(sent_msgs_after, received_msgs)

<class '__main__.InMemoryBroker'>.__enter__
[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched __init__() called()
[INFO] __main__: AIOKafkaProducer patched __init__() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
<class '__main__.InMemoryBroker'>.__exit__
[INFO] __main__: InMemoryBroker